In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statistics as st
from collections import Counter
sns.set(style="darkgrid")

# [Nomor 1](http://)

Berdasarkan analisa sekilas 5 data pertama dalam dataset, dapat dilihat bahwa beberapa column memiliki missing values. Selanjutnya, kita akan melakukan analisa tipe data serta jumlah data yang terdapat di dalam dataset.

In [ ]:
df = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
df.head()

Berdasarkan analisa menggunakan `.info()`, dapat kita lihat terdapat 307511 data di dalam dataset ini. Selanjutnya, saya memberikan alias kepada 2 column yang akan dianalisa dengan nama `amt_income` dan `amt_credit` agar memudahkan kita untuk mengakses column tersebut pada saat visualisasi.

In [ ]:
df.info()

In [ ]:
amt_income = df['AMT_INCOME_TOTAL']
amt_cred = df['AMT_CREDIT']

Dengan menggunakan fungsi bawaan yaitu `describe()`, kita mendapatkan summary statistics mengenai 2 column tersebut. Dapat kita lihat bahwa column `AMT_INCOME_TOTAL` memiliki rentang minimum dan maximum yang sangat jauh yaitu 25650 dan 117000000 serta rentang yang sangat tinggi antara Q3 (75%) dan nilai max. Hal ini dapat menjadi tanda bahwa ada kemungkinkan outlier di column ini. Sedangkan untuk column `AMT_CREDIT` juga memiliki rentang yang jauh walaupun tidak sejauh `AMT_INCOME_TOTAL`

In [ ]:
df[['AMT_INCOME_TOTAL', 'AMT_CREDIT']].describe().transpose()

Saya membuat beberapa fungsi tambahan untuk menganalisa dispersion, central tendency, dan untuk membuat visualisasi.

In [ ]:
def central_tendency(column):
  data = {}
  data['Column'] = column.name
  data['Mean'] = st.mean(column)
  data['Mode'] = Counter(column).most_common()[0][0]
  data['Median'] = st.median(column)
  return pd.DataFrame([data])

def dispersion(column):
  data = {}
  data['Column'] = column.name
  data['Variance'] = st.variance(column)
  data['Standard Deviation'] = st.stdev(column)
  data['Skew'] = column.skew()
  return pd.DataFrame([data])

def plot_distribution(column):
  f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize=(10, 6),
                                      gridspec_kw={"height_ratios": {0.2, 1}})
  
  info = central_tendency(column)
  mean = info['Mean'].values[0]
  median = info['Median'].values[0]
  mode = info['Mode'].values[0]

  sns.boxplot(x=column, ax=ax_box)
  ax_box.axvline(mean, color='r', linestyle='--')
  ax_box.axvline(median, color='g', linestyle=':')
  ax_box.axvline(mode, color='b', linestyle='-')
  ax_box.set(xlabel='')

  sns.histplot(x=column, ax=ax_hist, kde=True)
  ax_hist.axvline(mean, color='r', linestyle='--')
  ax_hist.axvline(median, color='g', linestyle=':')
  ax_hist.axvline(mode, color='b', linestyle='-')

  plt.legend({'Mean':mean, 'Median':median, 'Mode':mode})
  plt.show()

Dalam menganalisa central tendency, kita dapat fokus ke nilai mode dimana kita bisa lihat bahwa ternyata terdapat sangat banyak nilai `AMT_CREDIT` yang berada di rentang minimum. Setelah analisa central tendency, kita akan lanjutkan dengan analisa dispersion untuk mengecek skewness dari column-column tersebut.

In [ ]:
pd.concat([central_tendency(amt_income), central_tendency(amt_cred)], ignore_index=True)

Dari analisa dispersion, sebuah column dapat tergolong dalam normal distribution jika nilai skew berada dalam rentang -1 sampai 1. Melalui analisa di bawah, kita dapat melihat bahwa `AMT_INCOME_TOTAL` dan `AMT_CREDIT` bukanlah normal distribution dan memiliki outlier. Hal ini dapat dibuktikan lebih lanjut melalui plotting.

In [ ]:
pd.concat([dispersion(amt_income), dispersion(amt_cred)], ignore_index=True)

Melalui analisa distribusi, dapat kita lihat jumlah outlier yang sangat banyak.

In [ ]:
plot_distribution(amt_income)

Analisa `AMT_CREDIT` juga menunjukkan outliers yang lumayan banyak. Dapat dilihat juga di dalam boxplot, cukup banyak data point yang berada di batas atas Q3 + 1.5 * IQR.

In [ ]:
plot_distribution(amt_cred)

# Nomor 2

In [ ]:
days_emp = df['DAYS_EMPLOYED']

Pertama-tama, kita dapat menggunakan analisa `.describe()` untuk melihat ringkasan statistik mengenai data tersebut. Dapat kita lihat bahwa jarak Q3 ke nilai maksimum lumayan jauh sehingga ada kemungkinkan bahwa column ini memiliki outlier di atas Q3.

In [ ]:
df[['DAYS_EMPLOYED']].describe().transpose()

Setelah menggunakan analisa `.describe().`, kita dapat menlanjutkannya dengan analisa dispersion untuk menganalisa nilai skewness dari column `DAYS_EMPLOYED`. Dapat kita lihat bahwa nilai skewness berada di atas angka 1 sehingga dapat kita simpulkan bahwa kolum ini memang memiliki outlier di atas Q3. Kesimpulan ini kemudian dapat kita buktikan dengan menggunakan plotting histrogram serta box plot.

In [ ]:
dispersion(days_emp)

In [ ]:
plot_distribution(days_emp)

Berikut adalah fungsi untuk mencari semua data yang berada di bawah Q1-1.5 dikali IQR dan semua data yang berada di atas Q3+1.5 dikali IQR. Semua data tersebut akan ditampung di dalam list bernama `outliers`.

In [ ]:
def find_outlier(column):
  Q1 = column.quantile(0.25)
  Q3 = column.quantile(0.75)
  IQR = Q3 - Q1
  outliers = []

  for data in column:
    if (data > (Q3 + 1.5 * IQR)) | (data < (Q1 - 1.5 * IQR)):
      outliers.append(data)

  return outliers

Dapat kita lihat terdapat lumayan banyak outlier di dalam column ini. Kita dapat mencoba untuk menghapus keseluruhan outlier dan mengecek distribusinya. Dengan analisa `shape` sebelum dan setelah dihapusnya outlier, kita juga akan melihat jumlah outlier yang berhadil dihapus menggunakan metode ini. 

In [ ]:
outliers = find_outlier(days_emp)
outliers

Dengan menganalisa `shape`, kita dapat melihat bahwa terdapat 72217 outliers di column ini. Kita juga sudah membuat sebuah dataframe baru bernama `df_cleaned` dimana semua outlier tersebut sudah kita hapus. Setelah ini, kita dapat mencoba untuk melihat distribusi dari dataframe yang baru untuk memastikan bahwa semua outliers sudah kita hapus dari column `DAYS_EMPLOYED`.

In [ ]:
df_cleaned = df[~df['DAYS_EMPLOYED'].isin(outliers)]
print('Before removing outliers:', df.shape)
print('After removing outliers:', df_cleaned.shape)

Dapat kita lihat bahwa sekarang skewness sudah berada di posisi -0.95 yang artinya kita berhasil menghapus hampir semua outlier di dalam kolum ini. Kita juga bisa lihat bahwa distribusinya sudah menyerupai distribusi normal.

In [ ]:
dispersion(df_cleaned['DAYS_EMPLOYED'])

In [ ]:
plot_distribution(df_cleaned['DAYS_EMPLOYED'])

# Nomor 3

Melalui analisa statistik di atas, dapat kita lihat bahwa dataset ini memiliki data dyang berupa kumpulan nilai negatif. Untuk jarak antara kuartil tidak terlalu jauh.

In [ ]:
df[['DAYS_BIRTH']].describe().transpose()

In [ ]:
db = df['DAYS_BIRTH']

Dengan melakukan analisa skewness, dapat kita lihat bahwa outlier hampir tidak ada di kolum ini dengan nilai skewness hampir mendekati nol. Hal ini dapat menjadi insight bahwa untuk kolum ini tidak ada anomali atau dengan kata lain sudah *clean*. Kita juga dapat melihat bahwa days birth yang semakin kecil (Q1 ke bawah) memiliki jumlah count yang semakin kecil juga dan semakin besar nilai days birth (Q3 ke atas) memiliki jumlah count yang semakin besar, hal ini merupakan salah satu ciri-ciri dari sebuah distribusi normal.

In [ ]:
dispersion(db)

In [ ]:
plot_distribution(db)

# Nomor 4

In [ ]:
def age_segmentation(age):
  year = abs(age)/360
  age20_25 = year[(year <= 25) & (year >= 20)]
  age26_30 = year[(year <= 30) & (year >= 26)]
  age31_35 = year[(year <= 35) & (year >= 31)]
  age36_40 = year[(year <= 40) & (year >= 35)]
  age41_45 = year[(year <= 45) & (year >= 41)]
  age45above = year[(year > 45)]

  x = ['20-25', '26-30', '31-35', '36-40', '41-45', '45+']
  y = [len(age20_25.values), len(age26_30.values), len(age31_35.values),
       len(age36_40.values), len(age41_45.values), len(age45above.values)]

  plt.figure(figsize=(14, 5))
  sns.barplot(x=x, y=y, palette="rocket")
  plt.title("Age Segmentation")
  plt.xlabel("Age")
  plt.ylabel("Number of Customer")
  plt.show()

In [ ]:
df_failed = df[df['TARGET'] == 1]

Dapat kita lihat dari grafik berikut bahwa mayoritas customer yang memiliki home credit adalah orang-orang dengan umur 45 ke atas. Hal ini tentunya juga berdampak terhadap peningkatan jumlah orang di kategori 45+. Oleh karena itu, insight yang bisa diambil adalah segmentasi pasar home credit mayoritas berada di rentang umur 45 tahun ke atas dan korelasi antara `DAYS_BIRTH` dan `TARGET` kurang bisa kelihatan karena segmentasi customer yang memang skewed.

In [ ]:
age_segmentation(df_failed['DAYS_BIRTH'])

# NOMOR 5

Pertama-tama, kita harus menghapus missing values terlebih dahulu karena kolum NAME_INCOME_TYPE perlu diencode menjadi angka.

In [ ]:
df_without_null = df.dropna()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_without_null['NAME_INCOME_TYPE'] = le.fit(df_without_null['NAME_INCOME_TYPE']).transform(df_without_null['NAME_INCOME_TYPE'])

In [ ]:
cols = df_without_null[['NAME_INCOME_TYPE', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY', 'TARGET']]

Pertama-tama sebelum melakukan perhitungan korelasi antar feature, alangkah baiknya kita melakukan visualisasi distribusi tiap featurenya untuk menentukan tipe korelasi yang paling cocok. Dapat kita lihat bahwa kolum `TARGET` memiliki outlier, kita juga dapat melihat kolum `REGION_RATING_CLIENT_W_CITY` yang sangat condong di angka 2.0.

In [ ]:
cols.hist(figsize=(12, 6))

Setelah menganalisa distribusi di atas, penggunaan Spearman sangat disarankan karena Spearman mampu menangkap korelasi antar variabel yang tidak mengikuti ditribusi normal. Hasilnya adalah DAYS_BIRTH memiliki korelasi paling tinggi dibandingkan column yang lain yaitu sebesar 0.044. Korelasi ini masih bisa dibilang rendah oleh karena itu, insight untuk company adalah dapat menerapkan algoritma feature importance untuk mencari lebih dalam mana fitur yang sebenarnya paling mempengaruhi target. Hal ini dikarenakan nilai 0.044 belum bisa dijadikan patokan untuk memprediksi nilai target.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(cols.corr(method='spearman'), annot=True)
plt.show()

# NOMOR 6

Dengan melihat average income dapat kita lihat bahwa income type tertinggi adalah businessman sedangkan yang terendah adalah unemployed. Hal ini masuk akal dikarenakan orang yang tidak memiliki pekerjaan tentunya incomenya paling rendah dibandingkan yang lain. Kita juga bisa melihat bahwa orang yang pensiun masih memiliki income yang sedikit lebih tinggi dibandingkan unemployed, hal ini bisa saja dikarenakan oleh uang pensiun yang diberikan oleh perusahaan ataupun pemerintah.

In [ ]:
top_income = df.groupby('NAME_INCOME_TYPE', as_index=False)['AMT_INCOME_TOTAL'].mean()
top_income = top_income.sort_values(by='AMT_INCOME_TOTAL', ascending=False)

plt.figure(figsize=(20, 6))
sns.barplot(x='NAME_INCOME_TYPE', y='AMT_INCOME_TOTAL', data=top_income)
plt.show()

Dapat dilihat bahwa education dengan income tertinggi adalah academic degree sedangkan yang ketiga adalah incomplete higher. Hal ini tentu saja masuk akal dikarenakan potensi seseorang untuk mendapatkan pekerjaan yang lebih baik sangat berbanding lurus dengan gelar yang ia miliki.

In [ ]:
top_edu = df.groupby('NAME_EDUCATION_TYPE', as_index=False)['AMT_INCOME_TOTAL'].mean()
top_edu = top_edu.sort_values(by='AMT_INCOME_TOTAL', ascending=False).head(3)

plt.figure(figsize=(20, 6))
sns.barplot(x='NAME_EDUCATION_TYPE', y='AMT_INCOME_TOTAL', data=top_edu)
plt.show()

Lima tipe pekerjaan (OCCUPATION_TYPE) yang memiliki rata-rata salary tertinggi!
Bisa dilihat bahwa manager memiliki pendapatan tertinggi. Hal ini sesuai dengan jabatannya, karena manager bisa tergolong sebagai pekerjaan yang high-level.

In [ ]:
top_jobs = df.groupby('OCCUPATION_TYPE', as_index=False)['AMT_INCOME_TOTAL'].mean()
top_jobs = top_jobs.sort_values(by='AMT_INCOME_TOTAL', ascending=False).head()

plt.figure(figsize=(20, 6))
sns.barplot(x='OCCUPATION_TYPE', y='AMT_INCOME_TOTAL', data=top_jobs)
plt.show()

Status perkawinan yang paling umum adalah status menikah dilanjuti oleh single. 

In [ ]:
top_fame_stats = df.groupby('NAME_FAMILY_STATUS', as_index=False).count()
top_fame_stats = top_fame_stats.sort_values(by='SK_ID_CURR', ascending=False).head(3)

plt.figure(figsize=(20, 6))
sns.barplot(x='NAME_FAMILY_STATUS', y='AMT_INCOME_TOTAL', data=top_fame_stats)
plt.show()